#### Let's first load the llama2 base model and deploy it

In [1]:
import json
import re
from pprint import pprint
from src import process_text, utils

import pandas as pd
import torch
from datasets import Dataset, load_dataset
from huggingface_hub import notebook_login
from peft import LoraConfig, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from trl import SFTTrainer
import logging

logging.basicConfig(level=logging.INFO)

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
#MODEL_NAME = "meta-llama/Meta-Llama-3-8B-Instruct"
MODEL_NAME = "meta-llama/Llama-2-7b-hf"

#### Download and extract data

In [2]:
dataset = load_dataset("Salesforce/dialogstudio", "TweetSumm")

DEFAULT_SYSTEM_PROMPT = """
                        Below is a conversation between a human and an AI agent. Write a summary of the conversation.
                        """.strip()

Using the latest cached version of the module from /home/zeus/.cache/huggingface/modules/datasets_modules/datasets/Salesforce--dialogstudio/1dc993c7bcf3cb3392bd580ffe241b1c3a9233c2c2829b84d047788f56b099f5 (last modified on Sat Apr 20 01:03:16 2024) since it couldn't be found locally at Salesforce/dialogstudio, or remotely on the Hugging Face Hub.


In [12]:
from langchain_community.callbacks.utils import flatten_dict

sample_dict = flatten_dict(dataset['train'][0])

for key in sample_dict:
    print(f"{key}:{sample_dict[key]}")

original dialog id:b065262210783596c1fe79466b8f8985
new dialog id:TweetSumm--train--1
dialog index:1
original dialog info:{"summaries": {"extractive_summaries": [[{"is_agent": false, "sentences": ["So neither my iPhone nor my Apple Watch are recording my steps/activity, and Health doesn\u2019t recognise either source anymore for some reason."]}, {"is_agent": true, "sentences": ["To start, can you tell us the software versions your iPhone and Apple Watch are running currently?"]}, {"is_agent": false, "sentences": ["@AppleSupport My iPhone is on 11.1.2, and my watch is on 4.1."]}], [{"is_agent": false, "sentences": ["So neither my iPhone nor my Apple Watch are recording my steps/activity, and Health doesn\u2019t recognise either source anymore for some reason."]}, {"is_agent": true, "sentences": ["To start, can you tell us the software versions your iPhone and Apple Watch are running currently?"]}], [{"is_agent": false, "sentences": ["So neither my iPhone nor my Apple Watch are recording

In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

for key in dataset['train'][0]:
    print(f"{key}:{sample_dict[key]}")
#print()

original dialog id:b065262210783596c1fe79466b8f8985
new dialog id:TweetSumm--train--1
dialog index:1
original dialog info:{"summaries": {"extractive_summaries": [[{"is_agent": false, "sentences": ["So neither my iPhone nor my Apple Watch are recording my steps/activity, and Health doesn\u2019t recognise either source anymore for some reason."]}, {"is_agent": true, "sentences": ["To start, can you tell us the software versions your iPhone and Apple Watch are running currently?"]}, {"is_agent": false, "sentences": ["@AppleSupport My iPhone is on 11.1.2, and my watch is on 4.1."]}], [{"is_agent": false, "sentences": ["So neither my iPhone nor my Apple Watch are recording my steps/activity, and Health doesn\u2019t recognise either source anymore for some reason."]}, {"is_agent": true, "sentences": ["To start, can you tell us the software versions your iPhone and Apple Watch are running currently?"]}], [{"is_agent": false, "sentences": ["So neither my iPhone nor my Apple Watch are recording

In [63]:
def flatten_and_print(value):
    if isinstance(value, str):
        v_list = text_splitter.split_text(value)
        for v in v_list:
            print(v)
    elif isinstance(value, list):
        for v in value:
            flatten_and_print(v)
    elif isinstance(value, dict):
        for k in value.keys():
            print(f"{k}:")
            flatten_and_print(value[k])
    else:
        print(f'{value}')

print(flatten_and_print(dataset['train'][0]['log']))
'''for key in dataset['train'][0]:
    value = dataset['train'][0][key]

    if type(value) is list:
        for v in value:
            print(f"{v}")
    else:        
        print(f"{key}:{dataset['train'][0][key]}")'''

turn id:
1
user utterance:
So neither my iPhone nor my Apple Watch are
recording my steps/activity, and Health doesn’t
recognise either source anymore for some reason.
Any ideas? https://t.co/m9DPQbkftD @AppleSupport
please read the above.
system response:
@135060 Let’s investigate this together. To start,
can you tell us the software versions your iPhone
and Apple Watch are running currently?
dialog history:
original user side information:
{}
original system side information:
{}
turn id:
2
user utterance:
@AppleSupport My iPhone is on 11.1.2, and my watch
is on 4.1.
system response:
@135060 Thank you. Have you tried restarting both
devices since this started happening?
dialog history:
<USER> So neither my iPhone nor my Apple Watch are
recording my steps/activity, and Health doesn’t
recognise either source anymore for some reason.
Any ideas? https://t.co/m9DPQbkftD @AppleSupport
please read the above. <SYSTEM> @135060 Let’s
investigate this together. To start, can you tell
us the softw

'for key in dataset[\'train\'][0]:\n    value = dataset[\'train\'][0][key]\n\n    if type(value) is list:\n        for v in value:\n            print(f"{v}")\n    else:        \n        print(f"{key}:{dataset[\'train\'][0][key]}")'

In [44]:

for k in dataset['train'][0]['log'][0].keys():
    value = dataset['train'][0]['log'][0][k]

    if type(value) == 'str' and len(value) > 50:
        print(f"{k}:")
        v_list = text_splitter.split_text(value)
        for v in v_list:
            print(v)
    else:
        print(f"{k}:{dataset['train'][0]['log'][0][k]}")

turn id:1
user utterance:So neither my iPhone nor my Apple Watch are recording my steps/activity, and Health doesn’t recognise either source anymore for some reason. Any ideas? https://t.co/m9DPQbkftD @AppleSupport please read the above.
system response:@135060 Let’s investigate this together. To start, can you tell us the software versions your iPhone and Apple Watch are running currently?
dialog history:
original user side information:{}
original system side information:{}


In [49]:
value = dataset['train'][0]['log'][0]['user utterance']

v_list = text_splitter.split_text(value)

for v in v_list:
    print(v)

So neither my iPhone nor my Apple Watch are
recording my steps/activity, and Health doesn’t
recognise either source anymore for some reason.
Any ideas? https://t.co/m9DPQbkftD @AppleSupport
please read the above.


In [33]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=0)
dict = text_splitter.split_text(dataset['train'][0]['original dialog info'])
'''for d in dict:
    print(f"{d}")'''

print(dataset['train'][0]['log'])

[{'turn id': 1, 'user utterance': 'So neither my iPhone nor my Apple Watch are recording my steps/activity, and Health doesn’t recognise either source anymore for some reason. Any ideas? https://t.co/m9DPQbkftD @AppleSupport please read the above.', 'system response': '@135060 Let’s investigate this together. To start, can you tell us the software versions your iPhone and Apple Watch are running currently?', 'dialog history': '', 'original user side information': '{}', 'original system side information': '{}'}, {'turn id': 2, 'user utterance': '@AppleSupport My iPhone is on 11.1.2, and my watch is on 4.1.', 'system response': '@135060 Thank you. Have you tried restarting both devices since this started happening?', 'dialog history': '<USER> So neither my iPhone nor my Apple Watch are recording my steps/activity, and Health doesn’t recognise either source anymore for some reason. Any ideas? https://t.co/m9DPQbkftD @AppleSupport please read the above. <SYSTEM> @135060 Let’s investigate t

In [3]:
dataset["train"] = process_text.process_dataset(dataset["train"])
dataset["validation"] = process_text.process_dataset(dataset["validation"])



INFO:root:Processing dataset...
INFO:root:Processing dataset...


In [3]:
print(dataset['train'])

Dataset({
    features: ['original dialog id', 'new dialog id', 'dialog index', 'original dialog info', 'log', 'prompt'],
    num_rows: 879
})


#### Evaluate model

In [4]:
llm2, tokenizer = utils.create_model_and_tokenizer(llm_model_name=MODEL_NAME)

INFO:root:Creating model meta-llama/Llama-2-7b-hf..


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
examples = []
for data_point in dataset["test"].select(range(5)):
    summaries = json.loads(data_point["original dialog info"])["summaries"][
        "abstractive_summaries"
    ]
    summary = summaries[0]
    summary = " ".join(summary)
    conversation = process_text.create_conversation_text(data_point)
    examples.append(
        {
            "summary": summary,
            "conversation": conversation,
            "prompt": process_text.generate_deployment_prompt(conversation),
        }
    )
test_df = pd.DataFrame(examples)
test_df

,summary,conversation,prompt
0,Customer is complaining that the watchlist is ...,user: My watchlist is not updating with new ep...,### Instruction: Below is a conversation betwe...
1,Customer is asking about the ACC to link to th...,"user: hi , my Acc was linked to an old number....",### Instruction: Below is a conversation betwe...
2,Customer is complaining about the new updates ...,user: the new update ios11 sucks. I can’t even...,### Instruction: Below is a conversation betwe...
3,Customer is complaining about parcel service ...,user: FUCK YOU AND YOUR SHITTY PARCEL SERVICE ...,### Instruction: Below is a conversation betwe...
4,The customer says that he is stuck at Staines ...,user: Stuck at Staines waiting for a Reading t...,### Instruction: Below is a conversation betwe...


In [6]:
def summarize(model, text: str):
    inputs = tokenizer(text, return_tensors="pt").to(DEVICE)
    inputs_length = len(inputs["input_ids"][0])
    with torch.inference_mode():
        outputs = model.generate(**inputs, max_new_tokens=256, temperature=0.0001)
    return tokenizer.decode(outputs[0][inputs_length:], skip_special_tokens=True)


example = test_df.iloc[0]
summary = summarize(llm2, example.prompt)

In [7]:
pprint(summary)

('\n'
 '    user: My watchlist is not updating with new episodes (past couple days). '
 'Any idea why?\n'
 "    agent: Apologies for the trouble, Norlene! We're looking into this. In "
 'the meantime, try navigating to the season / episode manually.\n'
 '    user: Tried logging out/back in, that didn’t help\n'
 '    agent: Sorry! 😔 We assure you that our team is working hard to '
 'investigate, and we hope to have a fix ready soon!\n'
 '    user: Thank you! Some shows updated overnight, but others did not...\n'
 '    agent: We definitely understand, Norlene. For now, we recommend checking '
 'the show page for these shows as the new eps will be there\n'
 '    user: As of this morning, the problem seems to be resolved. Watchlist '
 'updated overnight with all new episodes. Thank you for your attention to '
 'this matter! I love Hulu 💚\n'
 "    agent: Awesome! That's what we love to hear. If you happen to need "
 "anything else, we'll be here to support! 💚\n"
 '\n'
 '    ### Output:\n'
 